# use the first uberon id as the identifier.

In [12]:
import pandas as pd
import json

In [13]:
# Private Datasets:

unpublished_metadata = None
with open("data/metadata/hubmap_metadata_unpublished.json", "r") as f:
    unpublished_metadata = json.loads(f.read())

# Public Datasets:

published_metadata = None
with open("data/metadata/hubmap_metadata_published.json", "r") as f:
    published_metadata = json.loads(f.read())

In [14]:
len(published_metadata), len(unpublished_metadata)

(1546, 1772)

In [15]:

def json_tree(element, n=0, type_=False):
    if isinstance(element, dict):
        for k,v in element.items():
            print("     "*n, k, ": ", type(v).__name__)
            json_tree(v, n+1)
        

    elif isinstance(element, list):
        if len(element) > 0:
            temp = [ele for ele in element if isinstance(ele, dict) or isinstance(ele, list)]
            # max_ele = sorted(temp.items(), key=lambda item: len(item[1]))[-1]
            max_vals = 0
            max_item = {}
            for item in temp:
                if len(item) > max_vals:
                    max_vals = len(item)
                    max_item = item

            # if isinstance(element[0], dict) or isinstance(element[0], list):
            #     json_tree(element[0], n+1)
            # print("")

            json_tree(max_item, n+1)
            print("")

In [16]:
columns = [
    "group_name",
    "dataset_id",
    "data_type",
    "publication_status",
    "sample_id",
    "sample_category",
    "organ",
    "organ_UBERON_ID",
    "sample_RUI_data",
    "BG via Azimuth",
    "BP via OMAP",
    "High resolution H&E/PAS" 
]

In [17]:
published_metadata["d15a20b5c07c638e758a25bb320091e1"]

{'anatomy_0': ['body'],
 'anatomy_1': ['large intestine'],
 'ancestor_counts': {'entity_type': {'Donor': 1, 'Sample': 18}},
 'ancestor_ids': ['d2a488545c2cd6dbe2b65361c919114f',
  '1de4ad9bd516660043b8e190e8978045',
  'cb53dd247ec7e651161932fde83b4ca8',
  'd3ad6409d2ad37ef7dbcc90ccc199f68',
  '4b25513dd7416a43d51ba5c636038284',
  'a596becf59c13e14a6489fb97fcdd2c9',
  '6e861a55dc7d7df6e5c36e7e1edab1e4',
  'df96f02667e0a3b40332dcaa6a91f850',
  '3280ceaaee2c24e262d46a5f83edfe98',
  '816a5df8f39c357ea19f4522ac7a641f',
  'efbf777d8d36f03cebe957e02eb9f8fe',
  'c4bb21f3b70bd34554f8ae5b557c38d6',
  'abe01e040eecba80c655fbca3aba165a',
  '450e073aab976738e7cd478063b5a93d',
  '2c4ed25579a369af5ae8f7523038235f',
  '18f8af62b88f26644d83a966be666677',
  '513ceb231c3406e87abddcf7bae9b5aa',
  'c8d593faf4ec323150460803a4e62d90',
  '062bd84696d51e02f670f064879b562d'],
 'ancestors': [{'created_by_user_displayname': 'Daniel Cotter',
   'created_by_user_email': 'Daniel.cotter@stanford.edu',
   'created_tim

In [18]:
json_tree(published_metadata["d15a20b5c07c638e758a25bb320091e1"])

 anatomy_0 :  list

 anatomy_1 :  list

 ancestor_counts :  dict
      entity_type :  dict
           Donor :  int
           Sample :  int
 ancestor_ids :  list

 ancestors :  list
           created_by_user_displayname :  str
           created_by_user_email :  str
           created_timestamp :  int
           data_access_level :  str
           entity_type :  str
           group_uuid :  str
           hubmap_id :  str
           lab_tissue_sample_id :  str
           last_modified_timestamp :  int
           mapped_data_access_level :  str
           mapped_last_modified_timestamp :  str
           mapped_sample_category :  str
           portal_metadata_upload_files :  list
                     description :  str
                     filepath :  str

           protocol_url :  str
           rui_location :  str
           sample_category :  str
           specimen_type :  str
           submission_id :  str
           tissue_type :  str
           uuid :  str

 contacts :  list
 

In [31]:
published_metadata["d15a20b5c07c638e758a25bb320091e1"]['metadata']['metadata']['assay_type']

'LC-MS Bottom-Up'

```
columns = [
    "group_name",
    "dataset_id",
    "data_type",
    "publication_status",
    "sample_id",
    "sample_category",
    "organ",
    "organ_UBERON_ID",
    "sample_RUI_data",
    "BG via Azimuth",
    "BP via OMAP",
    "High resolution H&E/PAS" 
]
```

In [19]:
data_dict = {}

for uuid, data in published_metadata.items():
    hubmap_id = data.get("hubmap_id", "")
    data_dict[hubmap_id] = {}
    data_dict[hubmap_id]["uuid"] = uuid
    data_dict[hubmap_id]["group_name"] = data.get("group_name", "")
    data_dict[hubmap_id]["hubmap_id"] = data.get("hubmap_id", "")
    data_dict[hubmap_id]["data_types"] = data.get("data_types", "")
    data_dict[hubmap_id]["publication_status"] = data.get("status", "")
    data_dict[hubmap_id]["assay_type"] = data["metadata"]["metadata"]["assay_type"] if "metadata" in data['metadata'] else "" if "metadata" in data else "" 

    data_dict[hubmap_id]["ancestors_sample_id_list"] = [ancestor['metadata'].get("sample_id", "") for ancestor in data["ancestors"] if "metadata" in ancestor]
    data_dict[hubmap_id]["ancestors_lab_tissue_sample_id_list"] = [ancestor.get("lab_tissue_sample_id", "") for ancestor in data["ancestors"]]
    data_dict[hubmap_id]["origin_sample_id"] = data['origin_sample'].get("lab_tissue_sample_id", "")
    
    data_dict[hubmap_id]["ancestors_sample_category_list"] = [ancestor.get("sample_category", "") for ancestor in data["ancestors"]]
    data_dict[hubmap_id]["origin_sample_category"] = data['origin_sample'].get("sample_category", "")

    data_dict[hubmap_id]["organ"] = data["origin_sample"].get("organ", "")
    data_dict[hubmap_id]["mapped_organ"] = data["origin_sample"].get("mapped_organ", "")
    data_dict[hubmap_id]["organ_UBERON_IDs"] = [json.loads(ancestor['rui_location'])['ccf_annotations'] for ancestor in data["ancestors"] if "rui_location" in ancestor]
    
    data_dict[hubmap_id]["publication_status"] = data.get("status", "")


In [25]:
data_dict

{'HBM645.ZQSN.258': {'uuid': '146063814f6bd2b6546e950d0ca5f8c3',
  'group_name': 'Vanderbilt TMC',
  'hubmap_id': 'HBM645.ZQSN.258',
  'data_types': ['MALDI-IMS-neg'],
  'publication_status': 'Published',
  'ancestors_sample_id_list': ['VAN0012-RK-103', ''],
  'ancestors_lab_tissue_sample_id_list': ['', '122', '68250', ''],
  'origin_sample_id': '68250',
  'ancestors_sample_category_list': ['section', 'block', 'organ', ''],
  'origin_sample_category': 'organ',
  'organ': 'RK',
  'mapped_organ': 'Kidney (Right)',
  'organ_UBERON_IDs': [[]]},
 'HBM446.GCXM.753': {'uuid': '6e95d91da2870ffc0a473c23038fc5e1',
  'group_name': 'Vanderbilt TMC',
  'hubmap_id': 'HBM446.GCXM.753',
  'data_types': ['MALDI-IMS-pos'],
  'publication_status': 'Published',
  'ancestors_sample_id_list': ['VAN0012-RK-103', ''],
  'ancestors_lab_tissue_sample_id_list': ['', '122', '68250', ''],
  'origin_sample_id': '68250',
  'ancestors_sample_category_list': ['section', 'block', 'organ', ''],
  'origin_sample_category